# Pandas basic practice

Check pep8 for standards of coding

In [ ]:
import pandas as pd

%load_ext nb_black

In [ ]:
DATAPATH = "data/season-1819.csv"

### 0. Připravte si data

1. Stáhněte si soubor https://datahub.io/sports-data/english-premier-league/r/season-1819.csv
1. Umístěte jej do složky data

### 1. Načtěte data set `season-1819_csv.csv` a provedte následující úpravy:

1. načtěte pouze prvních 17 sloupců
1. zahoďte sloupce Div, FTR a HTR
1. sloupec Date převeďte na datetime
1. přejmenujte sloupce:
  * FTHG: FullTimeHomeGoals
  * FTAG: FullTimeAwayGoals
  * HTHG: HalfTimeHomeGoals
  * HTAG: HalfTimeAwayGoals
  * HS: HomeShots
  * AS: AwayShots
  * HST: HomeShotsTarget
  * AST: AwayShotsTarget
  * HF: HomeFauls
  * AF: AwayFauls

In [ ]:
### load the fist 17 columns from season-1819_csv.csv and drop Div, FTR, HTR columns
COLS_TO_REMOVE = ["Div", "FTR", "HTR"]

df_input = pd.read_csv(DATAPATH, usecols=range(17)).drop(columns=COLS_TO_REMOVE)

In [ ]:
DATAURL = "https://datahub.io/sports-data/english-premier-league/r/season-1819.csv"

pd.read_csv(DATAURL, usecols=range(17), parse_dates=["Date"]).drop(
    columns=["Div", "FTR", "HTR"]
).dtypes

In [ ]:
df_input.dtypes

* paths as constants
* works with URL as well

In [ ]:
### convert data in Date column to datetime
df_input["Date"] = pd.to_datetime(df_input["Date"])

In [ ]:
### rename the columns according to spec
rename_dict = {
    "FTHG": "FullTimeHomeGoals",
    "FTAG": "FullTimeAwayGoals",
    "HTHG": "HalfTimeHomeGoals",
    "HTAG": "HalfTimeAwayGoals",
    "HS": "HomeShots",
    "AS": "AwayShots",
    "HST": "HomeShotsTarget",
    "AST": "AwayShotsTarget",
    "HF": "HomeFauls",
    "AF": "AwayFauls",
}

df_input = df_input.rename(columns=rename_dict)

In [ ]:
df_input.head()

### 2. Zjistěte jaké datové typy mají všechny sloupečky, zda data set obsahuje nějaké NaN hodnoty a jaké jsou základní statistiky číselných sloupců.

In [ ]:
### check types of all columns
df_input.info()

In [ ]:
### Check number of NaNs per column
df_input.apply(lambda x: x.isna().sum())

In [ ]:
### Get summary statistics
df_input.describe().T

### 3. Zjistěte kolik různých týmů hraje anglickou ligu? Jaké týmy to jsou?

In [ ]:
### Get number of teams in the league

home_teams = df_input["HomeTeam"].unique()
away_teams = df_input["AwayTeam"].unique()

assert set(home_teams) == set(
    away_teams
), f"Sets of home teams and away teams are not equal"

f"Number of teams in the league is {len(home_teams)}."

### 4. Přidejte sloupečky `HomeShotAccuracy` a `AwayShotAccuracy`, které říkají jaké procento střeleckých pokusů (Shots) nakonec mířilo na bránu (ShotsTarget). Vizualizujte rozdělení hodnot obou sloupců.

In [ ]:
df_input.head()

In [ ]:
import numpy as np

In [ ]:
def get_accuracy(shots, shots_target):
    shots_nozeros = shots.replace(0, np.nan)

    return shots_target / shots_nozeros

In [ ]:
df_input = df_input.assign(
    **{
        "HomeShotAccuracy": get_accuracy(
            df_input["HomeShots"], df_input["HomeShotsTarget"]
        ),
        "AwayShotAccuracy": get_accuracy(
            df_input["AwayShots"], df_input["AwayShotsTarget"]
        ),
    }
)

In [ ]:
df_input["HomeShotAccuracy"].hist()
df_input["AwayShotAccuracy"].hist()

* check (im)mutable types in python
* single responsibility principle

### 5. Vytvořte sloupeček `FullTimeWinner`, který obsahuje:

- `H`, pokud zvítězil `HomeTeam`
- `A`, pokud zvítězil `AwayTeam`
- `D`, pokud zápas skončil remízou

**Poznámka: Zatím neznáme apply, ale známe loc. Použijte tedy šikovně loc.** 

In [ ]:
### approach with .loc
def get_full_time_winner(_df):
    _df["FullTimeWinner"] = np.nan

    _df.loc[_df["FullTimeHomeGoals"] < _df["FullTimeAwayGoals"], "FullTimeWinner"] = "A"

    _df.loc[
        _df["FullTimeHomeGoals"] == _df["FullTimeAwayGoals"], "FullTimeWinner"
    ] = "D"

    _df.loc[_df["FullTimeHomeGoals"] > _df["FullTimeAwayGoals"], "FullTimeWinner"] = "H"

    return _df


### approach with mapping
# def get_full_time_winner(_df):
#     home_away_full_time_goals_difference = df_input["FullTimeHomeGoals"] - df_input["FullTimeAwayGoals"]
#     full_time_winner_dict = {-1: 'A', 0: 'D', 1:'H'}

#     _df['FullTimeWinner'] = np.sign(home_away_full_time_goals_difference).map(full_time_winner_dict)

#     return _df

df_input = get_full_time_winner(df_input)

### 6. Řekněme, že nás zajímá tým West Ham. Zjistětě, kolik průměrně vsítí gólů v domácích zápasech, které vyhraje a kolik v domácích zápasech, které prohraje.

In [ ]:
df_input.loc[(df_input["HomeTeam"] == "West Ham")].groupby("FullTimeWinner")[
    ["FullTimeHomeGoals"]
].mean().drop(["D"], axis=0).rename(
    {"A": "WestHamHomeLoss", "H": "WestHamHomeWin"}, axis=0
).rename(
    {"FullTimeHomeGoals": "FullTimeHomeGoalsMean"}, axis=1
).rename_axis(
    None, axis=0
)

### 7. Jaký je median faulů, střel na branku a vstřelených gólů West Ham pro předchozích pět domácích zápasů? Jak se tyto statistiky vyvíjí v čase? Vizualizujte je.

In [ ]:
df_input.head()

In [ ]:
df_west_ham_rolling_5_home_matches_median = (
    df_input.loc[
        ((df_input["HomeTeam"] == "West Ham")),
        ["FullTimeHomeGoals", "HomeShotsTarget", "HomeFauls"],
    ]
    .rolling(5)
    .median()
)

In [ ]:
df_west_ham_rolling_5_home_matches_median.iloc[-1].rename(
    "Last5HomeMatchesMedian", axis=0
).to_frame()

In [ ]:
df_west_ham_rolling_5_home_matches_median.plot()

### 8. Je domácí prostředí výhodou?

**Poznámka: Součástí řešení je zamyslet se, jakou formou tuto otázku zodpovědět a jaké předpoklady si mohu dovolit udělat.** 

### 9. Kolik který rozhodčí odpískal za sezónu průměrně faulů za zápas? Seřaďte je od těch nejpřísnějších.

### 10. (BONUS) Budu víc vyhrávat, když budu víc faulovat?

In [ ]:
# Discuss the solution and implement it with the students